<h1>Regression - Analysis data by ANN<h1>

In [18]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt 
from tqdm import tqdm

First, we load in our dataframes in our notebook. \
"train_x" is our attributes and "train_y" is our the true values for training part. \
"test_x" is our attributes and "test_y" is our the true values for testing part.

In [5]:
train_x = pd.read_csv("../2023-10-05_jennifer_data_preparation/independent_train.csv")
train_y = pd.read_csv("../2023-10-05_jennifer_data_preparation/dependent_train.csv")
test_x = pd.read_csv("../2023-10-05_jennifer_data_preparation/independent_test.csv")
test_y = pd.read_csv("../2023-10-05_jennifer_data_preparation/dependent_test.csv")
train_x.head(5)

,age,education-num,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving
0,-0.596105,-1.436812,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.029636,1.854382,0.038178,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.209024,0.867024,0.038178,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.255474,-0.120334,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.494133,0.867024,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_y.head(5)

,income
0,0
1,1
2,0
3,0
4,1


In [7]:
test_x.head(5)

,age,education-num,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving
0,1.339301,-1.765931,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.571549,-0.449454,1.821682,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.952220,1.525263,0.929930,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.184468,-2.095050,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.952220,-0.120334,0.038178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test_y.head(5)

,income
0,0
1,0
2,1
3,0
4,0


In [17]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(256, 25)
(256, 1)
(64, 25)
(64, 1)


In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [36]:
class ANN_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_hidden_layers):
        super(ANN_Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_hidden_layers = num_hidden_layers
        self.flatten = nn.Flatten()
        
        self.input_layer = nn.Linear
        self.hidden_layers = nn.ModuleList([
            nn.Linear(self.hidden_dim, self.hidden_dim) for _ in range(self.num_hidden_layers)
        ])
        self.output_layer = nn.Sigmoid()
    
    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x
    

In [39]:
input_dim = 25
hidden_dim = 256
num_hidden_layers = 5

model = ANN_Model(input_dim, hidden_dim, num_hidden_layers).to(device)
print(model)

ANN_Model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (hidden_layers): ModuleList(
    (0-4): 5 x Linear(in_features=256, out_features=256, bias=True)
  )
  (output_layer): Sigmoid()
)


In [42]:
num_epochs = 10
input_dim = 25
hidden_dim = 64
num_of_h = 10


generalization_errors_list = []

train_data, test_data = train_x, train_y
train_labels, test_labels = test_x, test_y

criterion = nn.CrossEntropyLoss()

for hidden_layers in tqdm(range(num_of_h)):
    fold_errors = []
    
    model = ANN_Model(input_dim, hidden_dim, hidden_layers).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(train_data)
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_data:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total
        fold_errors.append(1 - accuracy)
    
    
    generalization_error = sum(fold_errors) / len(fold_errors)
    generalization_errors_list.append(generalization_error)


best_structure_index = generalization_errors_list.index(min(generalization_errors_list))
best_structure = hidden_layer_structures[best_structure_index]

# print the result
print("Generalization Errors:")
for i, error in enumerate(generalization_errors_list):
    print(f"Hidden Layers: {hidden_layer_structures[i]}, Generalization Error: {error:.4f}")

print("Best Hidden Layer Structure:", best_structure)

# plot the diagrams
hidden_layer_count = [len(structure) for structure in hidden_layer_structures]
plt.plot(hidden_layer_count, generalization_errors_list, marker='o')
plt.xlabel("Number of Hidden Layers")
plt.ylabel("Generalization Error")
plt.title("Generalization Error vs. Number of Hidden Layers")
plt.xticks(hidden_layer_count)
plt.grid()
plt.show()


  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: optimizer got an empty parameter list